### Load backend

In [ ]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result
from mynd.runtime import Environment, load_environment

DOCUMENT: Path = Path("/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz")
result: Result[Environment, str] = load_environment()

match result:
    case Ok(environment):
        logger.info("loaded environment")
    case Err(message):
        logger.error(message)

result: Result[str, str] = backend.load_project(DOCUMENT)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = ["r23685bc_20100605_021022"]
target_chunks: list[Metashape.Chunk] = [
    chunk for chunk in document.chunks if chunk.label in target_labels
]

### Define stereo workflow

In [ ]:
from mynd.backends.metashape.data_types import StereoGroup

from mynd.backends.metashape.camera_helpers import (
    compute_stereo_calibration,
    get_stereo_groups,
)

from mynd.backends.metashape.image_helpers import (
    ImagePairLoader,
    generate_image_loaders,
)

from mynd.geometry.stereo import (
    StereoHomography,
    StereoCalibration,
    RectificationResult,
)

from mynd.geometry.stereo import (
    compute_rectifying_homographies,
    compute_rectifying_pixel_maps,
    compute_stereo_rectification,
)

import mynd.geometry.image_transformations as image
import mynd.geometry.stereo as stereo

def generate_stereo_rectification(
    chunk: Metashape.Chunk,
) -> tuple[RectificationResult, list[ImagePairLoader]]:
    """Returns rectification results and image loaders for the stereo groups in a Metashape chunk."""

    # Backend - Get pairs of sensors and cameras
    group: StereoGroup = get_stereo_groups(chunk)[0]

    # Backend - Compute the stereo calibration for the sensor pair
    calibration: StereoCalibration = compute_stereo_calibration(group.sensor_pair)

    # Backend - Set up image loaders for all camera pairs
    image_loaders: list[ImagePairLoader] = generate_image_loaders(group.camera_pairs)
    


    # TODO: Compute homographies
    homographies: StereoHomography = compute_rectifying_homographies(calibration)
    
    
    # NOTE: Test calculations of distorted image corners
    image_size: tuple[int, int] = (calibration.master.width, calibration.master.height)
    
    old_corners: tuple = stereo._get_image_corners(
        homographies.master,                 # homography
        calibration.master.camera_matrix,    # camera matrix
        image_size,                          # image dimensions - widht, height
        calibration.master.distortion,       # distortion coefficients
    )

    # NOTE: Test calculations of distorted image corners
    new_corners: ImageCorners = image.get_image_corners(
        homographies.master,
        calibration.master.camera_matrix,
        image_size,
        calibration.master.distortion,
    )

    logger.info(f"Old image corners: {old_corners}")
    logger.info(f"New image corners: {new_corners}")

    
    # TODO: Compute pixel maps
    rectification: RectificationResult = compute_rectifying_pixel_maps(calibration, homographies)

    # TODO: Compute image corners
    
    
    
    # Compute homographies, pixel maps and camera matrices for stereo rectification
    rectification: RectificationResult = compute_stereo_rectification(calibration)

    return rectification, image_loaders

### Define helper functionality to visualize forward and inverse image mapping

In [ ]:
from collections.abc import Callable

import cv2
import numpy as np
import tqdm

from mynd.geometry.image_transformations import PixelMap, invert_pixel_map, remap_image_pixels
from mynd.visualization.image_visualizers import WindowHandle, KeyCode, create_image_visualizer 


def update_window(
    handle: WindowHandle,
    key: KeyCode,
    callbacks: dict[KeyCode, Callable],
) -> None:
    """Updates a window by handling key events."""

    if not key in callbacks:
        return

    callback: Callable = callbacks.get(key)
    callback(handle, key)
    

def inspect_rectification_results(rectification: RectificationResult, image_loaders: list[ImagePairLoader]) -> None:
    """TODO"""

    forward_map: PixelMap = rectification.master.pixel_map
    inverse_map: PixelMap = invert_pixel_map(forward_map, iterations=20, step_size=0.5)

    windows = {
        "original": create_image_visualizer("original", width=500, height=600),
        "forwarded": create_image_visualizer("forwarded", width=500, height=600),
        "inversed": create_image_visualizer("inversed", width=500, height=600),
    }

    for loader in tqdm.tqdm(image_loaders, desc="Loading images..."):
        
        images: ImagePair = loader()

        forwarded: Image = remap_image_pixels(images.first, forward_map)
        inversed: Image = remap_image_pixels(forwarded, inverse_map)

        cv2.imshow(windows.get("original").name, images.first.to_array())
        cv2.imshow(windows.get("forwarded").name, forwarded.to_array())
        cv2.imshow(windows.get("inversed").name, inversed.to_array())

        key: int = cv2.waitKey(0)

        callbacks: dict = {
            KeyCode.SPACE: lambda window, key: None, # TODO: Load next
            KeyCode.ESC: lambda window, key: None, # TODO: Quit
        }

        # Update window
        update_window(windows.get("original"), key=key, callbacks=callbacks)

        # TODO: Figure out how to handle exit
        match key:
            case KeyCode.ESC:
                break

    cv2.destroyAllWindows()

for chunk in target_chunks:
    rectification, image_loaders = generate_stereo_rectification(chunk)
    inspect_rectification_results(rectification, image_loaders)